# 内存的调用和管理

python中对底层内存的调用和管理是很成熟的,基本的底层内存管理靠内置类型`memoryview`和标准库`array`的`array`类型来实现,在需要更细致的内存结构时使用`numpy`的`dnarray`来实现;而内存管理方面则通过gc回收.Cython作为python的补充依然沿用这套体系,仅是提供了一些额外的工具让我们可以进一步扩展其能力.

在Cython中对象的生命周期和垃圾回收通常依然受Python解释器的管理,Cython本身并不提供自己的垃圾回收机制或内存管理.这意味着Cython中创建的对象遵循Python的垃圾回收机制,只要没有对它们的引用它们就会被gc作为垃圾回收.也就是说正常情况下我们只需要需要像在普通Python代码一样处理即可.

但如果我们在Cython中涉及了底层的C/C++内存管理,比如使用了`malloc`分配内存,那么我们就必须自行负责管理这些内存块的释放.

In [1]:
%load_ext Cython

## [+]手动内存管理

`malloc()`,`realloc()`,`free()`是C中我们的老朋友了.在Cython中我们同样可以使用他们.需要注意这些操作必须有始有终,否则很容易造成内存泄漏.

其中`malloc()`,`realloc()`,`free()`都可以在包`libc.stdlib`中导入使用(`new`和`del`是C++中的关键字,Cython语法原生支持,但**纯净模式无法支持`new`和`del`的使用**)

`malloc()`,`realloc()`,`free()`的签名如下:

```cython
void* malloc(size_t size)
void* realloc(void* ptr, size_t size)
void free(void* ptr)
```

用法也很简单,就是要注意回收.

```python
import cython
from cython.cimports.libc.stdlib import malloc, free
...
data = cython.declare(
    cython.pointer(cython.double),
    cython.cast(
        cython.pointer(cython.double),
        malloc(cython.sizeof(cython.double) * 10)
    )
)
...
if not data:
    raise MemoryError()
...
free(data)
```

我们也可以在Python堆上分配内存,这种方式通常比上面的更好,因为它们提供的内存实际上是在Python的内部存储器管理系统中解决的.它们还对较小的内存块进行了特殊优化,从而通过避免昂贵的操作系统调用来加快其分配,因此这种方式效率更高.我们可以使用专用的接口`PyMem_Malloc`, `PyMem_Realloc`, `PyMem_Free`替换上面三个.
它们可以在`cpython.mem`标准声明文件中找到,用法和上面的也是一致的:

```cython
import cython
from cython.cimports.cpython.mem import PyMem_Malloc, PyMem_Realloc, PyMem_Free
...
data = cython.declare(
    cython.pointer(cython.double),
    cython.cast(
        cython.pointer(cython.double),
        PyMem_Malloc(cython.sizeof(cython.double) * 10)
    )
)
...
if not data:
    raise MemoryError()
...
PyMem_Free(data)
```

### 习惯用法

这种无论如何需要收尾的操作最常见的写法就是使用`try...finally`语法

> `try...finally`语法

In [3]:
%%cython
# distutils: extra_compile_args=-Wno-unreachable-code

import random
import cython
from cython.cimports.libc.stdlib import malloc, free

@cython.cfunc
def random_noise(number: cython.int=1)->object:
    i: cython.int
    # allocate number * sizeof(double) bytes of memory
    my_array: cython.pointer(cython.double) = cython.cast(
        cython.pointer(cython.double),
        malloc(cython.sizeof(cython.double) * number)
    )
    
    if not my_array:
        raise MemoryError()
    print("malloc ok")
    try:
        ran = random.normalvariate
        for i in range(number):
            my_array[i] = ran(0,1)

        return [ my_array[i] for i in range(number) ]
    finally:
        # return the previously allocated memory to the system
        free(my_array)
        print("free ok")
print(random_noise(10))

malloc ok
free ok
[0.6345041590850905, -0.5849516545667786, 2.070433489147209, -0.06019713722746313, 1.0983460236103788, 0.2639336208721861, -0.8315676788841333, -0.6957744634896049, 0.5910316826084278, -0.5295566472293454]


如果一块内存需要比可以由`try...finally`块管理的更长的生命周期.另一个有用的习惯是将其生命周期与Python对象相结合,以利用Python运行时的内存管理

> `cdef class`方法

```cython
import cython
from cython.cimports.cpython.mem import PyMem_Malloc, PyMem_Realloc, PyMem_Free

@cython.cclass
class SomeMemory:
    
    data: cython.pointer(cython.double)
    
    def __cinit__(self, number: cython.size_t)->None:
        # allocate some memory (uninitialised, may contain arbitrary data)
        self.data = cython.cast(
        cython.pointer(cython.double),
            PyMem_Malloc(cython.sizeof(cython.double) * number)
        )
        if not self.data:
            raise MemoryError()

    def resize(self, new_number: cython.size_t):
        # Allocates new_number * sizeof(double) bytes,
        # preserving the current content and making a best-effort to
        # re-use the original data location.
        mem = cython.cast(
        cython.pointer(cython.double),
            PyMem_Realloc(self.data, cython.sizeof(cython.double) * new_number)
        )
        if not mem:
            raise MemoryError()
        # Only overwrite the pointer if the memory was really reallocated.
        # On error (mem is NULL), the originally memory has not been freed.
        self.data = mem

    def __dealloc__(self):
        PyMem_Free(self.data)     # no-op if self.data is NULL  
```

## `array`内存列表

直接使用malloc这类工具相当于手动挡,适合进行一些精细操作.如果是一些没那么精细的操作还是用一些可以自动回收的工具更方便.

主要的工具就是`array`内存列表和内存视图

我们知道在C中array表示一段特定长度特定类型的连续内存,而array的变量实际保存的也是这段内存中第一位的指针.因此array也是直接操作内存的一个最常见的工具;python本身也提供了一个一维数组array对象用于提供高效数据处理;cython中也提供了对python中array的支持.


### C的array类型的变量声明和使用

和c中类似,cython中C的array类型也是同构定长的,声明方式也是类似`元素类型[长度]`的形式,也支持多维数组

```cython
a = cython.declare(cython.int[5]) # 长度为5,元素类型为int的一维数组
b = cython.declare(cython.int[2][2]) # 两行两列的元素类型为int的二维数组
```

使用上也和C中类似,可以使用下标指定位置读取或修改元素,但相比C中我们可以使用`for...in`语法遍历array中的元素,这就优雅很多了.

In [6]:
%%cython
# distutils: extra_compile_args=-Wno-unreachable-code
import cython

b = cython.declare(cython.int[2][2],[[1,2],[3,4]])

for i in b:
    for j in i:
        print(j)

1
2
3
4


### python中的array

Python有一个内置一维数组的原始类型的动态数组模块[array](https://docs.python.org/zh-cn/3.10/library/array.html).array是同构不定长的紧凑数组,使用一段连续内存保存c类型的数据.它身定位就是python中的高效的一维数组对象.

它支持的类型包括:

类型码|C类型|Python 类型|以字节表示的最小尺寸
---|---|---|---
'b'|signed char|int|1
'B'|unsigned char|int|1
'u'|wchar_t|Unicode 字符|2
'h'|signed short|int|2
'H'|unsigned short|int|2
'i'|signed int|int|2
'I'|unsigned int|int|2
'l'|signed long|int|4
'L'|unsigned long|int|4
'q'|signed long long|int|8
'Q'|unsigned long long|int|8
'f'|float|float|4
'd'|double|float|8

在创建时我们需要指定类型码,并可以附带一个可选的初始化项.

In [4]:
import array
a = array.array('i', [1, 2, 3])
print(a[0])

1


标准库array的操作是高效的且符合python使用习惯,而且既不用手动创建回收也不需要导入额外的第三方库就可以使用,因此是python中管理内存的一个通用方法.

但它也存在问题--只能是一维数据.这个其实很致命,因为很多计算任务需要矩阵,张量这样的多维数据,只有一维就相当局限了.因此在社区中更多的时候我们都会使用`numpy`的`ndarray`来代替标准库array.`numpy`的介绍我们在[别的篇章]中有介绍这里就不做赘述了.

### Cython扩展标准库`array`

Cython针对Python标准库`array`进行了扩展,让其可以在cython中被当做C对象使用的同时增加一些额外的操作.扩展方式就是在`import array`之前先导入`cpython.array`.

`array`被扩展后会有一个`data`字段用于指代这段内存,它可以执行如下操作:

+ `data.as_voidptr`,将这段内存中的内容强制转换为空指针
+ `data.as_chars`,将这段内存中的内容强制转换为`char`类型
+ `data.as_schars`,将这段内存中的内容强制转换为`schar`类型
+ `data.as_uchars`,将这段内存中的内容强制转换为`uchar`类型
+ `data.as_shorts`,将这段内存中的内容强制转换为`short`类型
+ `data.as_ushorts`,将这段内存中的内容强制转换为`ushort`类型
+ `data.as_ints`,将这段内存中的内容强制转换为`int`类型
+ `data.as_uints`,将这段内存中的内容强制转换为`uint`类型
+ `data.as_longs`,将这段内存中的内容强制转换为`long`类型
+ `data.as_ulongs`,将这段内存中的内容强制转换为`ulong`类型
+ `data.as_longlongs`,将这段内存中的内容强制转换为`longlong`类型
+ `data.as_ulonglongs`,将这段内存中的内容强制转换为`ulonglong`类型
+ `data.as_floats`,将这段内存中的内容强制转换为`float`类型
+ `data.as_doubles`,将这段内存中的内容强制转换为`double`类型
+ `data.as_pyunicodes`,将这段内存中的内容强制转换为python的`unicode`,即`str`类型

同时`array`增加了如下C部分静态方法

+ `int resize(array self, Py_ssize_t n) except -1`快速调整array大小.不适合重复的小增量增加操作.

+ `int resize_smart(array self, Py_ssize_t n) except -1`针对小增量增加容量操作优化过的调整array大小的方法.

+ `cdef inline array clone(array template, Py_ssize_t length, bint zero)`给定一个模板数组快速创建一个新数组.类型将与模板相同.如果为零,则将使用零初始化新数组.

+ `cdef inline array copy(array self)`复制一个数组

+ `cdef inline int extend_buffer(array self, char* stuff, Py_ssize_t n) except -1` 使用`stuff`对数组进行追加操作,`n`为追加的元素数而不是字节数

+ `cdef inline int extend(array self, array other) except -1`用另一个同构数组中的元素扩展当前数组

+ `cdef inline void zero(array self)`将数组内容全部置0

这些接口就需要使用cimports方式导入使用了

In [7]:
%%cython
# distutils: extra_compile_args=-Wno-unreachable-code
import cython
from cython.cimports.cpython import array
import array

a = cython.declare(array.array ,array.array('i', [1, 2, 3]))
print(a.data.as_ints[0])

# from cpython cimport array
# import array
b = cython.declare(array.array, array.array('u', ["这","是","啥"]))
print(b.data.as_pyunicodes[0])

c = cython.declare(array.array, array.array('i', [4, 5, 6]))
# extend a with b, resize as needed
array.extend(a, c)
print(a)
# resize a, leaving just original three elements
array.resize(a, len(a) - len(c))

1
这
array('i', [1, 2, 3, 4, 5, 6])


## `memoryviews`内存视图

内存视图是一段连续内存的一种抽象,通常用于处理大型数据集,如数组,二进制数据,图像等以提高性能和减少内存开销.python和cython中分别有自己的实现,相对而言python的内存视图比较简单,能做的比较少,而cython的则强大的多,cython的内存视图可以更加方便的和numpy的交互,这个后面会有专门的章节来介绍.

内存视图的作用是可以让我们以不同的方式查看和操作内存中的数据,而不需要复制数据,无论这个数据原本是什么类型什么形状,只要它可以被赋值给内存视图,我们都可以通过统一的接口直接操作其内容的二进制数据.

### python中的内存视图

Python的内置类型`memoryview`是一种高效的数据访问方式,它允许你直接访问底层数据的内存表示而无需复制数据.

In [6]:
# 创建一个字节数组
data = bytearray(b'Hello, World!')

# 创建一个内存视图，解释数据为ASCII字符串
view = memoryview(data)

# 修改内存视图中的数据
view[0] = ord('h')  # 将 'H' 修改为 'h',`ord`用于将字符转换为对应整数,内存视图的每一位是一个字节

# 输出修改后的数据
print(data.decode())  # 输出 "hello, World!"

hello, World!


可以看到,和标准库`array`一样,Python的内置类型`memoryview`是一个一维的对象,

### cython中的内存视图

cython中也可以定义内存视图,但需要注意,这俩虽然功能类似但并不是同一个东西.在cython中使用内存视图可像下面这样申明:

```python
view1D = cython.declare(cython.int[:])

view3D = cython.declare(cython.int[:,:,:])
```

内存视图可以申明维度,每个维度的声明用一个`,`隔开,这就是内存视图的基本声明形式

如果是在函数中声明内存视图类型的形参,其基本形式也是类似的

```python
from typing import Optional

@cython.cfunc
def process_3d_buffer(view: Optional[cython.int[:,:,:]]=None):
    ...
```

当使用纯python模式时，默认情况下会拒绝定义为内存视图的形参为None值,只有当类型声明为`Optional`时才允许

**注意**: cython中的内存视图是一个Python对象,可以直接被python解释器识别

### 内存布局

除了数据元素的指针外,Cython内存视图支持几乎所有导出Python新样式缓冲区接口的对象.如果内存必须为外部导入或代码优化的特定格式,那么了解或指定内存布局可能很有用.

内存布局包括两个方面

+ 数据访问,可分为:
    + 直接访问(direct),即无指针访问,每个索引值都明确表示数组中的一个元素.这意味着你可以使用整数索引来访问数组中的元素,而且索引值是连续的.
    + 间接访问(indirect),即指针访问,索引值并不直接对应于数组中的元素,而是引用其他数据结构或方式来确定元素的位置.这可以包括使用映射,布尔掩码,字符串键或其他非整数值来确定元素的位置.
+ 数据打包,即数据在内存中的保存方式,可分为:
    + 连续模式(contiguous),数组的元素是紧密排列的没有间隔,这意味着在数组的任何维度上相邻的元素在内存中是相邻的没有额外的间隔或步幅.连续数组通常在访问时效率更高,因为元素在内存中是紧凑存储的,可以更快地进行连续读取.
    + 跨步模式(strided),数组的不同维度之间的元素之间有一定的间隔,在这种情况下数组的元素不是紧密排列的,而是根据步幅跳跃地存储在内存中因此可能会导致访问效率较低.每个维度都有一个步幅值,表示在该维度上相邻元素之间的间隔.如果是跨步模式则可以使用步幅来识别每个维度连续索引所需的内存步数

#### 基本概念

NumPy数组为跨步直接数据访问提供了一个很好的模型,Cython参考了这一设计,我们可以使用它来复习C和Fortran连续数组以及数据跨步的概念.

最简单的数据布局可能是C连续数组.这也是NumPy和Cython数组中的默认布局.C连续意味着阵列数据在存储器中是连续的,并且阵列的第一维度中的相邻元素在存储器中相距最远,而最后维度中的邻近元素距离最近.例如在NumPy中

In [7]:
import numpy as np
arr = np.array([['0', '1', '2'], ['3', '4', '5']], dtype='S1')

`arr[0, 0]`和`arr[0，1]`在存储器中相距一个字节,而`arr[0, 0]`和`arr[1，0]`相距3个字节.这使我们产生了大步前进的想法.数组的每个轴都有一个步长,即从该轴上的一个元素到下一个元素所需的字节数.在上述情况下轴0和1的步长显然为

In [8]:
arr.strides

(3, 1)

对于3维连续数组,则有

In [10]:
c_contig = np.arange(24, dtype=np.int8).reshape((2,3,4))
c_contig.strides

(12, 4, 1)

对于`Fortran`中的连续数组则具有相反的内存顺序,即第一个轴上的元素在内存中最接近.

In [11]:
f_contig = np.array(c_contig, order='F')
np.all(f_contig == c_contig)

True

In [12]:
f_contig.strides

(1, 2, 6)

连续数组是指单个连续内存块包含数组元素的所有数据的数组,因此内存块长度是数组中元素数量和元素大小(字节)的乘积.在上面的例子中内存块是`2*3*4*1`字节长，其中`1`是`int8`的长度.

数组可以是连续的,而不是C或Fortran顺序

In [13]:
c_contig.transpose((1, 0, 2)).strides

(4, 12, 1)

对NumPy数组进行切片可以很容易地使其不连续

In [14]:
sliced = c_contig[:,1,:]
sliced.strides

(12, 1)

In [15]:
sliced.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

#### 默认行为

回到Cython中,一个正常默认的内存视图声明如下

```cython
my_memoryview = cython.declare(cython.int[:, :, :], obj)
```

它默认数据访问是直接方式(direct)数据打包为跨步模式(strided).

#### 指定C/Fortran顺序

我们可以使用语法`::1`及它所在的位置来指定内存视图是C或Fortran顺序的内存布局.

```cython
c_contiguous = cython.declare(cython.int[:, :, ::1], c_contig
f_contiguous = cython.declare(cython.int[::1, :, :], f_contig
```

当然,默认的numpy数组就是C顺序的内存布局,直接使用`cython.int[:,:,:]`也可以

如果我们希望将一个已有的内存视图中的内容放入指定C/Fortran顺序内存布局的视图中则可以使用`copy`方式

```cython
c_contiguous = cython.declare(cython.int[:, :, ::1], myview.copy()) # C顺序
f_contiguous_slice = cython.declare(cython.int[::1, :, :], myview.copy_fortran()) # Fortran顺序
```

#### 自定义内存布局

上面介绍的都是通用内存布局,但当需要进行定制化时,我们就需要借助`cython.view`中的几个常量了:

+ `generic`, 数据访问:直接访问(direct)或间接访问(indirect),数据打包:跨步模式(strided)

+ `strided`, 数据访问:直接访问(direct),数据打包:跨步模式(strided),这也是默认模式

+ `indirect`,数据访问:间接访问(indirect),数据打包:跨步模式(strided)

+ `contiguous`,数据访问:直接访问(direct),数据打包:连续模式(contiguous)

+ `indirect_contiguous`,数据访问:间接访问(indirect),数据打包:连续模式(contiguous)

指定自定义内存布局使用类似如下的声明方式:

```cython
cython.declare(cython.int[:, :, ::view.contiguous])
```

这些常数只能指定所在的维度的内存布局,这里有几个特殊规则

+ 只能设置**第一个维度**,**最后一个维度**,或**设置数据访问为间接访问(indirect)的维度后面一个维度**的数据打包为**连续模式(contiguous)**

### cython内存视图的赋值

内存视图最基础的赋值方式如下:

```cython
x = cython.declare(cython.int[:], exporting_object)
```


其中`exporting_object`可以是:

+ 从C中的array,比如

    ```cython
    exporting_object = cython.declare(cython.int[3][3][3])
    x = cython.declare(cython.int[:,:,:], exporting_object)
    ```
    
    
+ 从numpy中的ndarray,比如:

    ```cython
    import numpy as np
    narr = np.arange(27, dtype=np.dtype("i")).reshape((3, 3, 3))
    narr_view = cython.declare(cython.int[:,:,:], narr)
    ```
    

+ 从Cython中的array

    ```cython
    from cython.view import array
    cyarr = array(shape=(3, 3, 3), itemsize=sizeof(int), format="i")
    cyarr_view = cython.declare(cython.int[:,:,:], cyarr)
    ```
    
    
+ 从Python的array,注意这种只能赋值一维视图

    ```cyhton
    from cython.cimports.cpython import array
    import array
    a = cython.declare(array.array, array.array('i', [1, 2, 3]))
    ca = cython.declare(cython.int[:], a)
    ```
    

我们可以通过修改每一个维度的设置来控制赋值的规则,比如

```cython
partial_view = cython.declare(cython.int[1:102:2,:], exporting_object)
```

这表示exporting_object会按2D视图将缓冲区的第一维度限制为从第二个(索引1)开始的100行,然后个奇数行跳过的模式进行赋值.

每一个维度上都可以使用两个`:`的语法设置这哦规则,第一个`:`前表示起始位置,不填则表示0,两个`:`中间为结束位置,不填表示不限制;第二个`:`后表示间隔步长和规则,不填表示1.

### cython内存视图的索引

Cython内存视图上的索引访问将自动转换为内存地址.以下代码向其中请求一个二维内存视图的C类型的项目和索引：

```cython
buf = cython.declare(cython.int[:,:],exporting_object)

print(buf[1,2])
```

内存视图支持负数索引,其规则和python中一样.就是从末尾开始计数：

```cython
print(buf[-1,-2])
```

如果索引位数和内存视图维度不匹配,则会根据指定的索引返回一个新的视图.我们可以使用`...`来指代与我们想要的位置不相关的一堆位置从而避免写一堆`:,:`这样的东西

In [9]:
%%cython
# distutils: extra_compile_args=-Wno-unreachable-code
import cython
import numpy as np
narr = np.arange(27, dtype=np.dtype("i")).reshape((3, 3, 3))
narr_view = cython.declare(cython.int[:, :, :],narr)

print(narr_view[0,1,2])
print(narr_view[1].shape)
print(narr_view[1][0,0])
print(narr_view[:,:,1].shape)
print(narr_view[...,1][0,0])

5
[3, 3, 0, 0, 0, 0, 0, 0]
9
[3, 3, 0, 0, 0, 0, 0, 0]
1


### 扩充cython内存视图

我们可以通过为已有的内存视图增加`None`来扩展内存视图维度

```cython
myslice = cython.declare(cython.double[:],np.linspace(0, 10, num=50))

myslice[None, :]

# 2D array with shape (50, 1)
myslice[:, None]

# 3D array with shape (1, 10, 1)
myslice[None, 10:-20:2, None]

```

### 复制内存视图

内存视图可以通过赋值的形式进行复制

```cython
to_view = cython.declare(cython.int[:, :, :])
from_view= cython.declare(cython.int[:, :, :])
...

# copy the elements in from_view to to_view
to_view[...] = from_view
# or
to_view[:] = from_view
# or
to_view[:, :, :] = from_view

```

当然了如果满足要求也可以使用上面的`copy()`和`copy_fortran()`方法复制

### 内存视图的其他方法

Cython的内存视图对象(`cython.view.memoryview`)除了支持上面的操作外还具有以下属性:

+ `shape`: 形状
+ `strides`: 每个维度的步长
+ `suboffsets`: 子偏移量
+ `ndim`: 维度
+ `size`: 元素个数
+ `itemsize`/`nbytes`: 所有元素占用的字节数
+ `base`: 内存视图实际底层的对象
+ `T`: 转置后的矩阵

这些属性在python解释器中也可以被访问到.

In [10]:
%%cython
# distutils: extra_compile_args=-Wno-unreachable-code
import cython
import numpy as np

exporting_object = cython.declare(cython.int[3][3],[[1,2,3],[4,5,6],[7,8,9]])
narr_view = cython.declare(cython.int[:, :],exporting_object)
    
    
print(narr_view.shape)
print(narr_view.strides)
print(narr_view.ndim)
print(narr_view.size)
print(narr_view.base)
print(type(narr_view.base))
print(narr_view.T)


[3, 3, 0, 0, 0, 0, 0, 0]
[12, 4, 0, 0, 0, 0, 0, 0]
2
9
<class '_cython_magic_4dbcfddcdc82e110bbd0e3bc8b4107b795c2926f.array'>
<MemoryView of 'array' object>
